In [1]:
import sys
import numpy as np
from matplotlib import pyplot as plt
import xarray as xr

%matplotlib notebook

In [2]:
import qcodes as qc

In [8]:
from importlib import reload
from qctools import instruments as instools
reload(instools)

import labpythonconfig as cfg
reload(cfg)

from experiment import measurement
reload(measurement)
from experiment.measurement import Parameter, BaseMeasurement

In [9]:
station = qc.Station()

In [10]:
class DummyMeasurement(BaseMeasurement):
    def __init__(self, station):
        super().__init__(station)
        
        self.add_parameter('start', Parameter)
        self.add_parameter('stop', Parameter)
        self.add_parameter('npts', Parameter)
    
    def measure(self):
        # initialize the data we need
        xpoints = np.linspace(self.start(), self.stop(), self.npts())
        data = xr.DataArray(np.zeros_like(xpoints),
                            coords=[('xpoints', xpoints)])
        self.dataset['data'] = data
        self.save_data()
        
        # now populate in a primitive loop
        for i, x in enumerate(xpoints):
            
            # acquire some data, and save right away
            self.dataset['data'][i] = np.cos(x)
            self.save_data()                       
                    
        

m = DummyMeasurement(station)
m.start(0)
m.stop(1)
m.npts(11)
m.run()

In [12]:
fn = r"C:\data\2017-12\2017-12-17\2017-12-17_DummyMeasurement-#0001.hdf5"
with xr.open_dataset(fn) as ds:
    print(ds['data'])
ds.close()

<xarray.DataArray 'data' (xpoints: 11)>
array([ 1.      ,  0.995004,  0.980067,  0.955336,  0.921061,  0.877583,
        0.825336,  0.764842,  0.696707,  0.62161 ,  0.540302])
Coordinates:
  * xpoints  (xpoints) float64 0.0 0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.0


In [13]:
import h5py
f = h5py.File(fn, 'r')

In [18]:
for k, r in f['data'].attrs.items():
    print(k,r)

_FillValue [ nan]
DIMENSION_LIST [array([<HDF5 object reference>], dtype=object)]


In [22]:
f['data'].attrs['DIMENSION_LIST'][0][0]

<HDF5 object reference>